<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/04_Upload_frames_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #4: Upload frames to Zooniverse</h1>
<h3 align="right">Written by KSO Team</h3>

This notebook takes you through the process of: 
* Connecting to a Zooniverse project
* Retrieving the classifications/annotations that citizens have provided on the videos, using aggregation settings. To experiment with these aggregation settings, see Notebook 8. 
* Cutting the videos into images from the moment the species of interest was seen for the first time
* Modify these images with for example a color correction
* Upload the images to Zooniverse for the 2nd part of the workflow.

If you do not have a project with us yet, you can run the template project to get a taste of how it all works. Only the uploading of the images to Zooniverse will not be possible if you are not a member of our template project on Zooniverse.

🔴 <span style="color:red">&nbsp;NOTE: If you want to run another project than the template project, you need to have a Zooniverse account and be a member of the corresponding project.  </span>

# Set up and requirements

## Install kso_data_management and its requirements

In [ ]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    !pip install -q --upgrade pip
    !pip install -q -r kso-data-management/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys

    # Install requirements
    !pip install -q --no-warn-script-location --upgrade pip
    !pip install -qr ../requirements.txt

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.insert(0, "..")

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t4_utils as t4
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [ ]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

In [ ]:
connect = t_utils.ask_if_connect_zooniverse()

In [ ]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project, connect.result)

## Retrieve info about zooniverse clips

In [ ]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(
    project=project,
    db_info_dict=db_info_dict,
    zoo_project=zoo_project,
    zoo_info=["subjects", "classifications", "workflows"],
)

# Select the species of interest

In [ ]:
species_i = t4.choose_species(db_info_dict)

### Get the frames

In [ ]:
frames_to_upload_df = t4.get_frames(
    species_names=species_i.value,
    db_path=db_info_dict["db_path"],
    zoo_info_dict=zoo_info_dict,
    server_dict=db_info_dict,
    project=project,
    n_frames_subject=3,
    subsample_up_to=3,
)

## List available frames to upload to Zooniverse

### Frame modifications

In [ ]:
# Choose type of modifications
frame_modification = t3.clip_modification_widget()
frame_modification

In [ ]:
# Modify the clips
frames_ready_df = t4.modify_frames(
    frames_to_upload_df=frames_to_upload_df.df.reset_index(drop=True),
    species_i=species_i.value,
    modification_details=frame_modification.checks,
    project=project,
)

In [ ]:
# Review the size of the modified clips
t4.check_frame_size(frame_paths=frames_ready_df["modif_frame_path"].unique())

### Preview the frames

In [ ]:
# Compare the original and modified clips
t4.compare_frames(df=frames_ready_df)

### Set Zooniverse metadata

In [ ]:
# Create the metadata for the images
upload_to_zoo = t4.set_zoo_metadata(
    df=frames_ready_df,
    species_list=species_i.value,
    project=project,
    db_info_dict=db_info_dict,
)

### Upload frames to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

🔴 <span style="color:red">&nbsp;NOTE: If you run the template project without being a member of our template project, it is not possible to run this last cell.  </span>

In [ ]:
t4.upload_frames_to_zooniverse(
    upload_to_zoo=upload_to_zoo,
    species_list=species_i.value,
    db_info_dict=db_info_dict,
    project=project,
)

In [ ]:
# END